In [1]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import Markdown as md
from ipynb.fs.full import functions
import pandas as pd

InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
ticker_prices_filename = 'ticker_prices_transformed.xlsx'
ticker_prices_excel_book = pd.read_excel(io=ticker_prices_filename, header=[0,1], index_col=0, sheet_name=None)

market_prices_filename = 'market_prices_transformed.xlsx'
market_prices_excel_book = pd.read_excel(io=market_prices_filename, index_col=0, sheet_name=None)

In [3]:
price_column_name = 'price'

first_period_of_price = -10
last_period_of_price = 0

output_sheet_list = []

for case in market_prices_excel_book.keys():
    market_return = functions.calculate_market_return(market_price_df=market_prices_excel_book[case],
                                                      first_period_of_price=first_period_of_price, 
                                                      last_period_of_price=last_period_of_price)

    tickers_first_index = ticker_prices_excel_book[case].index.get_loc(first_period_of_price)
    tickers_last_index = ticker_prices_excel_book[case].index.get_loc(last_period_of_price)
    tickers_formation_prices = ticker_prices_excel_book[case].copy()[price_column_name].iloc[tickers_first_index:tickers_last_index+1]
    tickers_return = tickers_formation_prices.copy().ffill().pct_change()[1:]

    observation_df = functions.init_observation_df(tickers_return)
    observation_df = functions.calculate_u(observation_df, market_return)
    observation_df = functions.calculate_cu(observation_df)
    sorted_observation_df = functions.sort_by_last_formation_day(observation_df)

    output_sheet_list.append({
        'sheet_name': case,
        'data_frame': sorted_observation_df.copy(),
    })

    del market_return, tickers_first_index, tickers_last_index, tickers_formation_prices, tickers_return, \
        observation_df, sorted_observation_df



/home/adityamahesa/productivities/self-project/py-thesis/functions.ipynb:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  "outputs": [],
/home/adityamahesa/productivities/self-project/py-thesis/functions.ipynb:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  "outputs": [],
/home/adityamahesa/productivities/self-project/py-thesis/functions.ipynb:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at on

In [4]:
with pd.ExcelWriter('output_formation.xlsx') as excel_writer:
    for i in output_sheet_list:
        df = i['data_frame']
        df.index = df.index.map(str)
        sheet_name = i['sheet_name']
        df.to_excel(excel_writer=excel_writer, sheet_name=sheet_name)